# 10.1. Схожесть
Заполните пропуски в следующем коде, для того чтобы получилась система определения близости пользователей и объектов на основе [SVD-разложения](https://colab.research.google.com/drive/1ldt1uUd8bJwiuUHrp2xmTKjELYVLcdzZ#scrollTo=iB0F4K6Kxf2p), как было показано на лекции. 

Для этого требуется реализовать несколько функций, озаглавленных в приведённом ниже шаблоне класса. Функция `_get_svd()` должна выполнять svd-преобразование и возвращать матрицы P и Q (см. [лекцию](https://colab.research.google.com/drive/1aignzmtgTjC8HErXwUJzWlqDcmR8NdCg#scrollTo=LpPUNrmdXf0M)). Функция `get_similar_users` должна ранжировать всех пользователей по мере близости к искомому пользователю и возвращать индексы самых близких из них. Функция `get_similar_items` - аналогично, но по отношению к объектам. В качестве аргументов последние две функции принимают (1) количество похожих пользователей и объектов, индексы которых нужно вернуть, и (2) индекс текущего пользователя или объекта.

In [37]:
import numpy as np
from numpy.linalg import svd
from sklearn.neighbors import NearestNeighbors


class similaryty_analizer(object):
    def __init__(self, R: np.array):
        self.R = R
        self.n_users = R.shape[0]
        self.n_items = R.shape[1]

    def _get_svd(self, new_dim: int):
        R = self.R
        U, S, V = svd(R)

        U = U[:, :new_dim]
        S = np.diag(S[:new_dim])
        V = V[:new_dim, :]
        
        P = np.matmul(U, S)
        Q = V
        return P, Q

    def get_similar_users(self, n_users: int, user_id: int):
        P, Q = self._get_svd(self.n_users)

        nn = NearestNeighbors(n_neighbors=n_users+1)
        nn = nn.fit(P)

        user = P[user_id, :]
        neighbours = nn.kneighbors([user], return_distance=False)[:, 1:].ravel()
        return neighbours

    def get_similar_items(self, n_items: int, item_id: int):
        P, Q = self._get_svd(self.n_items)

        nn = NearestNeighbors(n_neighbors=n_items+1)
        nn = nn.fit(Q)

        item = Q[item_id, :]
        neighbours = nn.kneighbors([item], return_distance=False)[:, 1:].ravel()
        return neighbours

# 10.2. Социальная сеть "Друзья".

Представим, что Вы - основатель нового амбициозного стартапа "Социальная сеть: Друзья", предлагающего организовывать поиск новых друзей при помощи сравнения вкусов в области кинематографа. Каждому регистрирующемуся на портале пользователю предлагается заполнить очень простую анекету. Она состоит из N фильмов, для каждого из которых требуется поставить одну из двух оценок: 1, если пользователь может сказать, что указанный фильм ему нравится, и 0 иначе (если не нравится или пользователь его не смотрел).

После заполнения анкеты пользователь получает список наиболее подходящих ему с точки зрения кинематографической совместимости.

Ваша задача - написать класс `friendadviser`, в рамках которого реализовать следующие методы:

- Метод `fit(self, R)`, принимающий на вход матрицу `R` размерности $M × N$, где $M$ - число зарегистрированных пользователей вашей социальной сети, а $N$ - число фильмов в анкете. Элемент матрицы $r_{ij}$ - это отметка, поставленная пользователем $i$ в анкете напротив фильма $j$.
- `_sim(u1, u2)` - функция, вычисляющая [схожесть (PMI)](https://colab.research.google.com/drive/1aignzmtgTjC8HErXwUJzWlqDcmR8NdCg#scrollTo=XfDfevVjrQ_X) пользователей $u_1$ и $u_2$ по векторам их оценок. Рекомендуем использовать "усечённую" версию PMI, которая в лекции называется score.
- `U_idx(u0, alpha)` - функция, позволяющая найти набор зарегистрированных пользователей (а именно, их индексов), схожесть вкусов которых с новым пользователем $u_0$ не меньше значения alpha.
- `find_friends(u0, how_many)` - функция, подыскивающая для пользователя $u_0$ новых друзей в количестве, заданном аргументом `how_many`. На выходе мы ожидаем получить массив с индексами таких друзей.  Индексы ради удобства верните в порядке убывания схожести интересов.

In [180]:
import numpy as np

class friendadviser(object):
    def fit(self, R):
        self.R = R
        self.n_items = R.shape[0]
        self.n_users = R.shape[1]

    def _sim(self, u1, u2):
        return np.sum((self.R[:, u1] == self.R[:, u2]) & (self.R[:, u1] == 1)) / self.R[:, u1].sum()

    def U_idx(self, u0, alpha):
        sim = np.array([[u, s._sim(u0, u)] for u in range(self.n_users) if u != u0])
        sim = sim[sim[:, 1] > alpha, 0]
        sim = np.sort(sim)[::-1]
        return sim
    
    def find_friends(self, u0, how_many):
        sim = self.U_idx(u0, -1)
        return sim[:how_many]

In [181]:
r1 = [0, 1, 0, 1, 0]
r2 = [0, 1, 0, 1, 0]
r3 = [1, 0, 1, 1, 0]
r4 = [1, 0, 1, 1, 0]
r5 = [0, 1, 0, 1, 0]
r6 = [1, 0, 1, 1, 0]

R = np.array([r1, r2, r3, r4, r5, r6])

In [182]:
R[:, 1]

array([1, 1, 0, 0, 1, 0])

In [183]:
u1 = 0
u2 = 1

In [184]:
s = friendadviser()
s.fit(R)

In [185]:
s.U_idx(0, -1)

array([4., 3., 2., 1.])

In [186]:
s.find_friends(0, 2)

array([4., 3.])

In [123]:
s.U_idx(0, 0)[:, s.U_idx(0, 0)[:, 1] == 1]

array([[0.],
       [1.]])

In [91]:
np.sum((R[:, u1] == R[:, u2]) & (R[:, u1] == 1))

0

In [47]:
R

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 1],
       [1, 0, 1],
       [0, 1, 0],
       [1, 0, 1]])

In [45]:
films_liked_by_sveta = df[df['Света'] == 1]
n_sveta_liked = films_liked_by_sveta.shape[0]
P_pasha_cond_sveta = films_liked_by_sveta['Паша'].sum()/n_sveta_liked
P_pasha_cond_sveta

array([[0, 1, 0],
       [0, 1, 0],
       [1, 0, 1],
       [1, 0, 1],
       [0, 1, 0],
       [1, 0, 1]])